In [ ]:
import pandas as pd
import json
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [ ]:
def squad_json_to_dataframe_dev(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
#     ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
#     js['q_idx'] = ndx
    main = m[['id','question','context','answers']].set_index('id').reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [ ]:
# training data
input_file_path = '/content/drive/MyDrive/AIN311Project/data/train-v1.1.json'
record_path = ['data','paragraphs','qas','answers']
train = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path)
# dev data
input_file_path = '/content/drive/MyDrive/AIN311Project/data/dev-v1.1.json'
record_path = ['data','paragraphs','qas','answers']
verbose = 0
dev = squad_json_to_dataframe_dev(input_file_path=input_file_path,record_path=record_path)

Reading the json file
processing...


<ipython-input-2-378025e0a1eb>:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  js = pd.io.json.json_normalize(file , record_path )
<ipython-input-2-378025e0a1eb>:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  m = pd.io.json.json_normalize(file, record_path[:-1] )
<ipython-input-2-378025e0a1eb>:17: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  r = pd.io.json.json_normalize(file,record_path[:-2])
<ipython-input-2-378025e0a1eb>:24: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()


shape of the dataframe is (87599, 6)
Done
Reading the json file
processing...
shape of the dataframe is (10570, 5)
Done


In [ ]:
train = train[:250]
dev = dev[:150]

In [ ]:
train.rename(columns = {'text':'answers'}, inplace = True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [ ]:
train

,index,question,context,answer_start,answers,c_id
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",515,Saint Bernadette Soubirous,0
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",188,a copper statue of Christ,0
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",279,the Main Building,0
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",381,a Marian place of prayer and reflection,0
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",92,a golden statue of the Virgin Mary,0
...,...,...,...,...,...,...
245,5733ca05d058e614000b6264,What occurs at midnight preceding a football h...,"Football gameday traditions During home games,...",245,the Drummers' Circle,50
246,5733ca05d058e614000b6265,From where does the Band of the Fighting Irish...,"Football gameday traditions During home games,...",594,the steps of Bond Hall,50
247,5733ca05d058e614000b6266,What songs does the trumpet section of the Ban...,"Football gameday traditions During home games,...",480,the Notre Dame Victory March and the Notre Dam...,50
248,5733ca05d058e614000b6267,On what day do Notre Dame home football games ...,"Football gameday traditions During home games,...",424,Saturday,50


In [ ]:
dev.insert(4,"answerss", " ")
dev.insert(3,"answer_start", " ")
for i in range(len(dev["answers"])):
  dev["answerss"][i] = dev["answers"][i][0]["text"]
  dev["answer_start"][i] = dev["answers"][i][0]["answer_start"]
dev.drop('answers', inplace=True, axis=1)
dev.rename(columns = {'answerss':'answers'}, inplace = True)


<ipython-input-8-77d24bdfccd4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dev["answerss"][i] = dev["answers"][i][0]["text"]
<ipython-input-8-77d24bdfccd4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dev["answer_start"][i] = dev["answers"][i][0]["answer_start"]


In [ ]:
dev

,id,question,context,answer_start,answers,c_id
0,56be4db0acb8001400a502ec,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,177,Denver Broncos,0
1,56be4db0acb8001400a502ed,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,249,Carolina Panthers,0
2,56be4db0acb8001400a502ee,Where did Super Bowl 50 take place?,Super Bowl 50 was an American football game to...,403,"Santa Clara, California",0
3,56be4db0acb8001400a502ef,Which NFL team won Super Bowl 50?,Super Bowl 50 was an American football game to...,177,Denver Broncos,0
4,56be4db0acb8001400a502f0,What color was used to emphasize the 50th anni...,Super Bowl 50 was an American football game to...,488,gold,0
...,...,...,...,...,...,...
145,56bf21b43aeaaa14008c9526,What was the last Super Bowl that took place a...,"The league announced on October 16, 2012, that...",242,Super Bowl XLIV,6
146,56bf21b43aeaaa14008c9528,"In 2012, how many stadiums were named as final...","The league announced on October 16, 2012, that...",51,two,6
147,56bf21b43aeaaa14008c9529,What was the entity that stepped in and caused...,"The league announced on October 16, 2012, that...",508,Florida legislature,6
148,56d6ef6a0d65d21400198260,"Prior to this consideration, when did San Fran...","The league announced on October 16, 2012, that...",309,1985,6


In [ ]:
our_train = pd.read_csv("/content/drive/MyDrive/AIN311Project/data/Questions.csv")

In [ ]:
train.drop('index', inplace=True, axis=1)
train.drop('answer_start', inplace=True, axis=1)
train.drop('c_id', inplace=True, axis=1)

In [ ]:
train

,question,context,answers
0,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",Saint Bernadette Soubirous
1,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",a copper statue of Christ
2,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",the Main Building
3,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",a Marian place of prayer and reflection
4,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",a golden statue of the Virgin Mary
...,...,...,...
245,What occurs at midnight preceding a football h...,"Football gameday traditions During home games,...",the Drummers' Circle
246,From where does the Band of the Fighting Irish...,"Football gameday traditions During home games,...",the steps of Bond Hall
247,What songs does the trumpet section of the Ban...,"Football gameday traditions During home games,...",the Notre Dame Victory March and the Notre Dam...
248,On what day do Notre Dame home football games ...,"Football gameday traditions During home games,...",Saturday


In [ ]:
our_train

,context,question,answers
0,"“A friend in need is a friend indeed, that is ...",What is the definition of a true friend who wi...,friends
1,"“A friend in need is a friend indeed, that is ...",Who always share and support each other?,real friends
2,"“A friend in need is a friend indeed, that is ...",What do real friends feel when we are happy?,joyful
3,Artificial intelligence was founded as an acad...,In what decade did machine learning dominate t...,the first decades of the 21st century
4,Artificial intelligence was founded as an acad...,When was artificial intelligence founded?,1956
...,...,...,...
193,The 1948 Palestine war was fought in the terri...,Who used to rule palestine before the first wo...,ottoman empire
194,The 1948 Palestine war was fought in the terri...,when did the state of Israel was established,In May 1948
195,The 1948 Palestine war was fought in the terri...,who established Israel in May 1948?,Jewish Yishuv
196,The 1948 Palestine war was fought in the terri...,How many Arabs were displaced during the 1948 ...,700


In [ ]:
trainQ = pd.concat([train, our_train], ignore_index=True)
trainQ

,question,context,answers
0,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",Saint Bernadette Soubirous
1,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",a copper statue of Christ
2,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",the Main Building
3,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",a Marian place of prayer and reflection
4,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",a golden statue of the Virgin Mary
...,...,...,...
443,Who used to rule palestine before the first wo...,The 1948 Palestine war was fought in the terri...,ottoman empire
444,when did the state of Israel was established,The 1948 Palestine war was fought in the terri...,In May 1948
445,who established Israel in May 1948?,The 1948 Palestine war was fought in the terri...,Jewish Yishuv
446,How many Arabs were displaced during the 1948 ...,The 1948 Palestine war was fought in the terri...,700


In [ ]:
dev.drop('id', inplace=True, axis=1)
dev.drop('answer_start', inplace=True, axis=1)
dev.drop('c_id', inplace=True, axis=1)

In [ ]:
dev

,question,context,answers
0,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,Denver Broncos
1,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,Carolina Panthers
2,Where did Super Bowl 50 take place?,Super Bowl 50 was an American football game to...,"Santa Clara, California"
3,Which NFL team won Super Bowl 50?,Super Bowl 50 was an American football game to...,Denver Broncos
4,What color was used to emphasize the 50th anni...,Super Bowl 50 was an American football game to...,gold
...,...,...,...
145,What was the last Super Bowl that took place a...,"The league announced on October 16, 2012, that...",Super Bowl XLIV
146,"In 2012, how many stadiums were named as final...","The league announced on October 16, 2012, that...",two
147,What was the entity that stepped in and caused...,"The league announced on October 16, 2012, that...",Florida legislature
148,"Prior to this consideration, when did San Fran...","The league announced on October 16, 2012, that...",1985


In [ ]:
length_of_the_messagesQ = trainQ["context"].str.split("\\s+")
print("Max number of words for context = ", length_of_the_messagesQ.str.len().max())
print("Index = ", length_of_the_messagesQ.str.len().idxmax())

Max number of words for context =  319
Index =  261


In [ ]:
length_of_the_messagesQQ = trainQ["question"].str.split("\\s+")

print("Max number of words = ", length_of_the_messagesQQ.str.len().max())
print("Index = ", length_of_the_messagesQQ.str.len().idxmax())

Max number of words =  23
Index =  97


In [ ]:
trainQ.to_csv("/content/drive/MyDrive/AIN311Project/data/QuestionG/train.csv",index = False)
dev.to_csv("/content/drive/MyDrive/AIN311Project/data/QuestionG/valid.csv",index = False)

In [ ]:
trainA = trainQ.drop('question', inplace=False, axis=1)

In [ ]:
for i in range(len(trainA)):
  trainA["answers"][i] = trainA["answers"][i] + " : "

In [ ]:
trainA = trainA.groupby('context').sum().reset_index()

In [ ]:
trainA

,context,answers
0,A Science Hall was built in 1883 under the dir...,"Fr. Zahm : 1950 : Joseph LaFortune : 83,000 sq..."
1,About 80% of undergraduates and 20% of graduat...,80% : four : 15 : 20% : 14 :
2,All he could think about was how it would all ...,Uncertainty about his future : The end of some...
3,All of Notre Dame's undergraduate students are...,Learning Resource Center : five : The First Ye...
4,"Architecturally, the school has a Catholic cha...",Saint Bernadette Soubirous : a copper statue o...
...,...,...
89,"This Main Building, and the library collection...",Washington Hall : 1879 : Rev. William Corby : ...
90,"Throughout its history, the United States has ...",50 : United States of America : more than 400 ...
91,Wars are the biggest killers of humanity in th...,wars : a loss to humanity : World Wars I and I...
92,"World War I, often abbreviated as WWI or WW1, ...",28 july 1914 : 11 november 1918 : 9 million : ...


In [ ]:
trainA["answers"][40]

'mathematical : Math : mobile devices, computers, software, architecture ancient and modern, art, money, engineering and even sports : every civilized society : Dover Publications : more complex : the position of the sun and the physics of hunting : counting : '

In [ ]:
trainA["context"]

0     A Science Hall was built in 1883 under the dir...
1     About 80% of undergraduates and 20% of graduat...
2     All he could think about was how it would all ...
3     All of Notre Dame's undergraduate students are...
4     Architecturally, the school has a Catholic cha...
                            ...                        
89    This Main Building, and the library collection...
90    Throughout its history, the United States has ...
91    Wars are the biggest killers of humanity in th...
92    World War I, often abbreviated as WWI or WW1, ...
93    “A friend in need is a friend indeed, that is ...
Name: context, Length: 94, dtype: object

In [ ]:
length_of_the_messagesA = trainA["answers"].str.split("\\s+")
print("Max number of words = ", length_of_the_messagesA.str.len().max())

Max number of words =  92


In [ ]:
trainA.to_csv("/content/drive/MyDrive/AIN311Project/data/AnswerG/train.csv",index = False)